In [38]:
# Instalación de paquetes necesarios
%pip install ucimlrepo matplotlib seaborn pandas xgboost catboost imbalanced-learn -q

Note: you may need to restart the kernel to use updated packages.


In [39]:
from ucimlrepo import fetch_ucirepo

import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, confusion_matrix, 
    classification_report
)

from imblearn.over_sampling import SMOTE

In [36]:
# fetch dataset
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468)

ConnectionError: Error connecting to server

In [ ]:
# data (as pandas dataframes)
X = online_shoppers_purchasing_intention_dataset.data.features
y = online_shoppers_purchasing_intention_dataset.data.targets

NameError: name 'online_shoppers_purchasing_intention_dataset' is not defined

In [ ]:
# metadata
print(online_shoppers_purchasing_intention_dataset.metadata)

NameError: name 'online_shoppers_purchasing_intention_dataset' is not defined

In [ ]:
# variable information
print(online_shoppers_purchasing_intention_dataset.variables)

NameError: name 'online_shoppers_purchasing_intention_dataset' is not defined

In [40]:
# ALTERNATIVA: Cargar desde archivo local si fetch_ucirepo falla
import os

# Verificar si existe el archivo local
if os.path.exists('dataset/online_shoppers.csv'):
    data = pd.read_csv('dataset/online_shoppers.csv')
    
    # Separar features y target
    X = data.drop('Revenue', axis=1)
    y = data[['Revenue']]
    
    print(f"Dataset cargado")
    print(f"Shape: {data.shape}")
else:
    print("Error: No se encuentra el archivo 'dataset/online_shoppers.csv'")

Dataset cargado
Shape: (12330, 18)


# FASE 1: PREPARACIÓN DE DATOS

## Paso 1.1: División Train/Test (Estratificada)

Usamos una configuración de 80% entrenamiento y 20% test, de esta forma queda de la siguiente manera: 
- 9,864 train / 2,466 test
- Desbalance 85/15: ~370 casos positivos en test (suficiente para evaluación confiable)
- Suficientes datos para entrenar y aplicar SMOTE posteriormente

Además, con la estratificación se mantiene la proporción 85/15 en ambos conjuntos

In [41]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.20,           # Definimos el 20% para test
    random_state=42,          # Reproducibilidad
    stratify=y                # Mantiene proporción de clases
)

print(f"Tamaño de los conjuntos:")
print(f"Train: {X_train.shape[0]:,} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test:  {X_test.shape[0]:,} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\nDistribución de clases en TRAIN:")
train_counts = y_train['Revenue'].value_counts()
print(f"False (No compra): {train_counts[False]:,} ({train_counts[False]/len(y_train)*100:.2f}%)")
print(f"True (Compra):     {train_counts[True]:,} ({train_counts[True]/len(y_train)*100:.2f}%)")

print(f"\nDistribución de clases en TEST:")
test_counts = y_test['Revenue'].value_counts()
print(f"False (No compra): {test_counts[False]:,} ({test_counts[False]/len(y_test)*100:.2f}%)")
print(f"True (Compra):     {test_counts[True]:,} ({test_counts[True]/len(y_test)*100:.2f}%)")

Tamaño de los conjuntos:
Train: 9,864 muestras (80.0%)
Test:  2,466 muestras (20.0%)

Distribución de clases en TRAIN:
False (No compra): 8,338 (84.53%)
True (Compra):     1,526 (15.47%)

Distribución de clases en TEST:
False (No compra): 2,084 (84.51%)
True (Compra):     382 (15.49%)


## Paso 1.2: Codificación de variables

In [42]:
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# 1. WEEKEND: Bool → Int
X_train_encoded['Weekend'] = X_train_encoded['Weekend'].astype(int)
X_test_encoded['Weekend'] = X_test_encoded['Weekend'].astype(int)

# 2. MONTH: OneHot
month_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
month_encoded_train = month_encoder.fit_transform(X_train_encoded[['Month']])
month_encoded_test = month_encoder.transform(X_test_encoded[['Month']])
month_cols = [f'Month_{cat}' for cat in month_encoder.categories_[0][1:]]
month_train_df = pd.DataFrame(month_encoded_train, columns=month_cols, index=X_train_encoded.index)
month_test_df = pd.DataFrame(month_encoded_test, columns=month_cols, index=X_test_encoded.index)
X_train_encoded = pd.concat([X_train_encoded.drop('Month', axis=1), month_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop('Month', axis=1), month_test_df], axis=1)

# 3. VISITORTYPE: OneHot
visitor_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
visitor_encoded_train = visitor_encoder.fit_transform(X_train_encoded[['VisitorType']])
visitor_encoded_test = visitor_encoder.transform(X_test_encoded[['VisitorType']])
visitor_cols = [f'VisitorType_{cat}' for cat in visitor_encoder.categories_[0][1:]]
visitor_train_df = pd.DataFrame(visitor_encoded_train, columns=visitor_cols, index=X_train_encoded.index)
visitor_test_df = pd.DataFrame(visitor_encoded_test, columns=visitor_cols, index=X_test_encoded.index)
X_train_encoded = pd.concat([X_train_encoded.drop('VisitorType', axis=1), visitor_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop('VisitorType', axis=1), visitor_test_df], axis=1)

# 4. OPERATINGSYSTEMS: OneHot
os_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
os_encoded_train = os_encoder.fit_transform(X_train_encoded[['OperatingSystems']])
os_encoded_test = os_encoder.transform(X_test_encoded[['OperatingSystems']])
os_cols = [f'OS_{int(cat)}' for cat in os_encoder.categories_[0][1:]]
os_train_df = pd.DataFrame(os_encoded_train, columns=os_cols, index=X_train_encoded.index)
os_test_df = pd.DataFrame(os_encoded_test, columns=os_cols, index=X_test_encoded.index)
X_train_encoded = pd.concat([X_train_encoded.drop('OperatingSystems', axis=1), os_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop('OperatingSystems', axis=1), os_test_df], axis=1)

# 5. BROWSER: OneHot con Grouping (Top 5 + Other)
top_5_browsers = X_train_encoded['Browser'].value_counts().head(5).index.tolist()
X_train_encoded['Browser_grouped'] = X_train_encoded['Browser'].apply(
    lambda x: x if x in top_5_browsers else 99
)
X_test_encoded['Browser_grouped'] = X_test_encoded['Browser'].apply(
    lambda x: x if x in top_5_browsers else 99
)
browser_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
browser_encoded_train = browser_encoder.fit_transform(X_train_encoded[['Browser_grouped']])
browser_encoded_test = browser_encoder.transform(X_test_encoded[['Browser_grouped']])
browser_cols = [f'Browser_{int(cat) if cat != 99 else "Other"}' for cat in browser_encoder.categories_[0][1:]]
browser_train_df = pd.DataFrame(browser_encoded_train, columns=browser_cols, index=X_train_encoded.index)
browser_test_df = pd.DataFrame(browser_encoded_test, columns=browser_cols, index=X_test_encoded.index)
X_train_encoded = pd.concat([X_train_encoded.drop(['Browser', 'Browser_grouped'], axis=1), browser_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop(['Browser', 'Browser_grouped'], axis=1), browser_test_df], axis=1)

# 6. REGION: OneHot
region_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
region_encoded_train = region_encoder.fit_transform(X_train_encoded[['Region']])
region_encoded_test = region_encoder.transform(X_test_encoded[['Region']])
region_cols = [f'Region_{int(cat)}' for cat in region_encoder.categories_[0][1:]]
region_train_df = pd.DataFrame(region_encoded_train, columns=region_cols, index=X_train_encoded.index)
region_test_df = pd.DataFrame(region_encoded_test, columns=region_cols, index=X_test_encoded.index)
X_train_encoded = pd.concat([X_train_encoded.drop('Region', axis=1), region_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop('Region', axis=1), region_test_df], axis=1)

# 7. TRAFFICTYPE: Target Encoding
traffic_conversion_rate = X_train_encoded.join(y_train).groupby('TrafficType')['Revenue'].mean().to_dict()
global_mean = y_train['Revenue'].mean()
X_train_encoded['TrafficType_Encoded'] = X_train_encoded['TrafficType'].map(traffic_conversion_rate)
X_test_encoded['TrafficType_Encoded'] = X_test_encoded['TrafficType'].map(traffic_conversion_rate).fillna(global_mean)
X_train_encoded = X_train_encoded.drop('TrafficType', axis=1)
X_test_encoded = X_test_encoded.drop('TrafficType', axis=1)

print(f"Codificación completada")
print(f"X_train: {X_train_encoded.shape}")
print(f"X_test:  {X_test_encoded.shape}")

Codificación completada
X_train: (9864, 43)
X_test:  (2466, 43)


### Resumen de Codificación Completada

**Transformaciones aplicadas:**

| Variable Original | Estrategia | Columnas Generadas | Justificación |
|------------------|------------|-------------------|---------------|
| **Month** | OneHot (drop first) | 9 | Sin orden natural, captura estacionalidad |
| **VisitorType** | OneHot (drop first) | 2 | Solo 3 categorías nominales |
| **Weekend** | Bool → Int | 1 | Ya binaria, solo conversión |
| **OperatingSystems** | OneHot (drop first) | 7 | 8 valores manejables |
| **Browser** | OneHot + Grouping | 5 | Top 5 + "Other" (reducido de 13) |
| **Region** | OneHot (drop first) | 8 | 9 valores geográficos |
| **TrafficType** | Target Encoding | 1 | 20 valores → 1 numérica |

**Resultado:**
- Features originales: 17
- Features después de codificación: **43** (vs 73 con OneHot completo)
- Reducción de dimensionalidad: 41% menos features
- Todos los encoders ajustados SOLO con train

## Paso 1.3: Escalado de Variables Numéricas

Usaremos **RobustScaler** porque en el análisis exploratorio se vieron muchos outliers en variables de duración.

In [43]:
numerical_cols_to_scale = [
    'Administrative', 'Administrative_Duration',
    'Informational', 'Informational_Duration',
    'ProductRelated', 'ProductRelated_Duration',
    'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay',
    'TrafficType_Encoded'
]

X_train_scaled = X_train_encoded.copy()
X_test_scaled = X_test_encoded.copy()

scaler = RobustScaler()
X_train_scaled[numerical_cols_to_scale] = scaler.fit_transform(X_train_encoded[numerical_cols_to_scale])
X_test_scaled[numerical_cols_to_scale] = scaler.transform(X_test_encoded[numerical_cols_to_scale])

print(f"X_train_scaled: {X_train_scaled.shape}")
print(f"X_test_scaled:  {X_test_scaled.shape}")

X_train_scaled: (9864, 43)
X_test_scaled:  (2466, 43)


### Resumen de la fase 1: Preparación de Datos

| Paso | Acción |
|------|--------|
| 1.1 | División Train/Test (80/20 estratificado) |
| 1.2 | Codificación de variables categóricas |
| 1.3 | Escalado de variables numéricas (RobustScaler) |

**Datasets listos para entrenamiento:**
- `X_train_scaled`: 9,864 muestras × 43 features
- `X_test_scaled`: 2,466 muestras × 43 features
- `y_train`: 9,864 etiquetas (84.5% No compra, 15.5% Compra)
- `y_test`: 2,466 etiquetas (84.5% No compra, 15.5% Compra)

# FASE 2: MODELO BASELINE

## Paso 2.1: Entrenamiento de Modelos sin Balanceo

Entrenamos varios modelos con los datos desbalanceados (84.5% No compra / 15.5% Compra) para establecer una línea base, esto tambiem nos permitirá comparar el efecto de SMOTE posteriormente.

In [44]:
# Aplanar y_train y y_test para compatibilidad
y_train_flat = y_train.values.ravel()
y_test_flat = y_test.values.ravel()

# Diccionario para almacenar resultados
baseline_results = {}

# Modelos a entrenar
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=2000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42, early_stopping=True),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, random_seed=42, verbose=0)
}

for name, model in models.items():
    start_time = time.time()
    
    # Entrenar modelo
    model.fit(X_train_scaled, y_train_flat)
    
    # Predicciones
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Métricas
    accuracy = accuracy_score(y_test_flat, y_pred)
    precision = precision_score(y_test_flat, y_pred)
    recall = recall_score(y_test_flat, y_pred)
    f1 = f1_score(y_test_flat, y_pred)
    roc_auc = roc_auc_score(y_test_flat, y_pred_proba) if y_pred_proba is not None else None
    
    training_time = time.time() - start_time
    
    # Resultados
    baseline_results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'Training Time (s)': training_time,
        'Model': model
    }
    
    roc_auc_str = f"{roc_auc:.4f}" if roc_auc is not None else "N/A"

## Paso 2.2: Evaluación y Comparación de Resultados

In [45]:
results_df = pd.DataFrame(baseline_results).T
results_df = results_df[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'Training Time (s)']]
results_df = results_df.round(4)

print(results_df.to_string())

# Identificar mejor modelo por F1-Score (ya que es más apropiado para datos desbalanceados)
best_model_name = results_df['F1-Score'].idxmax()
print(f"\nMejor modelo (por F1-Score): {best_model_name}")
print(f"F1-Score: {results_df.loc[best_model_name, 'F1-Score']:.4f}")
print(f"ROC-AUC: {results_df.loc[best_model_name, 'ROC-AUC']:.4f}")

                     Accuracy Precision    Recall  F1-Score   ROC-AUC Training Time (s)
Logistic Regression  0.882401  0.755556  0.356021  0.483986  0.886398          1.916839
Decision Tree        0.856853  0.537662  0.541885  0.539765  0.728236          0.156197
Random Forest        0.896594  0.739623  0.513089  0.605873  0.918734          2.109679
SVM                   0.88159  0.692308  0.424084  0.525974  0.851294         10.530048
KNN                  0.886456  0.675862  0.513089  0.583333  0.848243          0.168999
MLP                  0.896188  0.699367  0.578534  0.633238   0.91351          4.958604
XGBoost              0.903082  0.721362  0.609948  0.660993  0.928317          0.296999
CatBoost              0.90146  0.723473  0.589005  0.649351   0.92913          0.612362

Mejor modelo (por F1-Score): XGBoost
F1-Score: 0.6610
ROC-AUC: 0.9283


In [46]:
# Matriz de confusión y reporte detallado del mejor modelo
best_model = baseline_results[best_model_name]['Model']
y_pred_best = best_model.predict(X_test_scaled)

print(f"Análisis de {best_model_name}")

# Matriz de confusión
cm = confusion_matrix(y_test_flat, y_pred_best)
print("\nMatriz de Confusión:")
print(f"                  Predicho: No Compra | Predicho: Compra")
print(f"Real: No Compra          {cm[0][0]:6d}     |     {cm[0][1]:6d}")
print(f"Real: Compra             {cm[1][0]:6d}     |     {cm[1][1]:6d}")

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test_flat, y_pred_best, target_names=['No Compra', 'Compra']))

Análisis de XGBoost

Matriz de Confusión:
                  Predicho: No Compra | Predicho: Compra
Real: No Compra            1994     |         90
Real: Compra                149     |        233

Reporte de Clasificación:
              precision    recall  f1-score   support

   No Compra       0.93      0.96      0.94      2084
      Compra       0.72      0.61      0.66       382

    accuracy                           0.90      2466
   macro avg       0.83      0.78      0.80      2466
weighted avg       0.90      0.90      0.90      2466



### Resumen FASE 2: Modelo Baseline

**Resultados obtenidos Sin balanceo de clases:**

| Modelo | Accuracy | Precision | Recall | F1-Score | ROC-AUC |
|--------|----------|-----------|--------|----------|---------|
| Random Forest | 0.8966 | 0.7396 | 0.5131 | 0.6059 | 0.9187 |
| KNN | 0.8865 | 0.6759 | 0.5131 | 0.5833 | 0.8482 |
| Logistic Regression | 0.8824 | 0.7556 | 0.3560 | 0.4840 | 0.8862 |
| SVM | 0.8816 | 0.6923 | 0.4241 | 0.5260 | 0.8513 |
| Decision Tree | 0.8569 | 0.5377 | 0.5419 | 0.5398 | 0.7282 |
| CatBoost | 0.8569 | 0.5377 | 0.5419 | 0.5398 | 0.7282 |
| **XGBoost** | **0.9030** | **0.7213** | **0.6099** | **0.6609** | **0.9283** |

**Análisis:**

1. XGBoost es el mejor modelo con F1=0.6609 y ROC-AUC=0.9283
2. Problema del desbalanceo es evidente:
   - Alta accuracy (90.3%) pero bajo recall (60.9%) para clase positiva
   - El modelo predice bien "No Compra" (97% recall) pero falla en "Compra" (51% recall)
   - 186 falsos negativos (casi la mitad de las compras no detectadas)

# FASE 3: SMOTE PROGRESIVO

## Paso 3.1: Aplicación de SMOTE Incremental

Se aplica SMOTE incrementalmente generando 5%, 10% y 15% adicional de muestras de la clase minoritaria. SMOTE se aplica SOLO en el conjunto de entrenamiento.

**Estado actual:**
- Clase minoritaria (Compra): 1,526 muestras (15.47%)
- Clase mayoritaria (No Compra): 8,338 muestras (84.53%)

In [47]:
# Estado actual de las clases en train
current_minority = (y_train_flat == 1).sum()  # 1526
current_majority = (y_train_flat == 0).sum()  # 8338
total_train = len(y_train_flat)

# Hacemos el Calculo sampling_strategy para cada nivel de SMOTE
# sampling_strategy = num_samples_minority / num_samples_majority

# SMOTE 5%
smote_5_samples = current_minority + int(total_train * 0.05)
strategy_5 = smote_5_samples / current_majority

# SMOTE 10%
smote_10_samples = current_minority + int(total_train * 0.10)
strategy_10 = smote_10_samples / current_majority

# SMOTE 15%
smote_15_samples = current_minority + int(total_train * 0.15)
strategy_15 = smote_15_samples / current_majority

print(f"\nSMOTE 5%:")
print(f"- Muestras minoritarias: {smote_5_samples} ({smote_5_samples/(current_majority+smote_5_samples)*100:.2f}%)")
print(f"- sampling_strategy: {strategy_5:.4f}")

print(f"\n SMOTE 10%:")
print(f"- Muestras minoritarias: {smote_10_samples} ({smote_10_samples/(current_majority+smote_10_samples)*100:.2f}%)")
print(f"- sampling_strategy: {strategy_10:.4f}")

print(f"\nSMOTE 15%:")
print(f"- Muestras minoritarias: {smote_15_samples} ({smote_15_samples/(current_majority+smote_15_samples)*100:.2f}%)")
print(f"- sampling_strategy: {strategy_15:.4f}")

# Se aplica SMOTE para cada nivel
smote_configs = {
    'SMOTE 5%': (strategy_5, smote_5_samples),
    'SMOTE 10%': (strategy_10, smote_10_samples),
    'SMOTE 15%': (strategy_15, smote_15_samples)
}

smote_datasets = {}

for name, (strategy, expected_samples) in smote_configs.items():
    smote = SMOTE(sampling_strategy=strategy, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train_flat)
    
    smote_datasets[name] = {
        'X': X_resampled,
        'y': y_resampled,
        'minority_count': (y_resampled == 1).sum(),
        'majority_count': (y_resampled == 0).sum(),
        'total': len(y_resampled)
    }
    
    print(f"\n{name}:")
    print(f"Total muestras: {smote_datasets[name]['total']}")
    print(f"Compra (1): {smote_datasets[name]['minority_count']} ({smote_datasets[name]['minority_count']/smote_datasets[name]['total']*100:.2f}%)")
    print(f"No Compra (0): {smote_datasets[name]['majority_count']} ({smote_datasets[name]['majority_count']/smote_datasets[name]['total']*100:.2f}%)")


SMOTE 5%:
- Muestras minoritarias: 2019 (19.49%)
- sampling_strategy: 0.2421

 SMOTE 10%:
- Muestras minoritarias: 2512 (23.15%)
- sampling_strategy: 0.3013

SMOTE 15%:
- Muestras minoritarias: 3005 (26.49%)
- sampling_strategy: 0.3604

SMOTE 5%:
Total muestras: 10357
Compra (1): 2019 (19.49%)
No Compra (0): 8338 (80.51%)

SMOTE 10%:
Total muestras: 10850
Compra (1): 2512 (23.15%)
No Compra (0): 8338 (76.85%)

SMOTE 15%:
Total muestras: 11343
Compra (1): 3005 (26.49%)
No Compra (0): 8338 (73.51%)

SMOTE 10%:
Total muestras: 10850
Compra (1): 2512 (23.15%)
No Compra (0): 8338 (76.85%)

SMOTE 15%:
Total muestras: 11343
Compra (1): 3005 (26.49%)
No Compra (0): 8338 (73.51%)


## Paso 3.2: Entrenamiento con SMOTE y Evaluación

Entrenaremos los 6 modelos con cada configuración de SMOTE y compararemos con el baseline.

In [48]:
smote_results = {}

models_smote = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=2000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42, early_stopping=True),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, random_seed=42, verbose=0)
}

# Entrenamos con cada configuración de SMOTE
for smote_name, smote_data in smote_datasets.items():
    print(f"{smote_name}")
    
    X_smote = smote_data['X']
    y_smote = smote_data['y']
    
    smote_results[smote_name] = {}
    
    for model_name, model in models_smote.items():
        start_time = time.time()
        
        # Entrenar
        model.fit(X_smote, y_smote)
        
        # Predecir en test (Recordar que el test es sin SMOTE)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Métricas
        accuracy = accuracy_score(y_test_flat, y_pred)
        precision = precision_score(y_test_flat, y_pred)
        recall = recall_score(y_test_flat, y_pred)
        f1 = f1_score(y_test_flat, y_pred)
        roc_auc = roc_auc_score(y_test_flat, y_pred_proba) if y_pred_proba is not None else None
        
        training_time = time.time() - start_time
        
        smote_results[smote_name][model_name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'ROC-AUC': roc_auc,
            'Training Time (s)': training_time
        }
        
        roc_auc_str = f"{roc_auc:.4f}" if roc_auc is not None else "N/A"
        print(f"  {model_name:20s} | F1: {f1:.4f} | Recall: {recall:.4f} | ROC-AUC: {roc_auc_str}")

SMOTE 5%
  Logistic Regression  | F1: 0.5240 | Recall: 0.4136 | ROC-AUC: 0.8876
  Decision Tree        | F1: 0.5181 | Recall: 0.5236 | ROC-AUC: 0.7162
  Logistic Regression  | F1: 0.5240 | Recall: 0.4136 | ROC-AUC: 0.8876
  Decision Tree        | F1: 0.5181 | Recall: 0.5236 | ROC-AUC: 0.7162
  Random Forest        | F1: 0.6372 | Recall: 0.5654 | ROC-AUC: 0.9166
  Random Forest        | F1: 0.6372 | Recall: 0.5654 | ROC-AUC: 0.9166
  SVM                  | F1: 0.5644 | Recall: 0.4817 | ROC-AUC: 0.8688
  KNN                  | F1: 0.5863 | Recall: 0.5733 | ROC-AUC: 0.8437
  SVM                  | F1: 0.5644 | Recall: 0.4817 | ROC-AUC: 0.8688
  KNN                  | F1: 0.5863 | Recall: 0.5733 | ROC-AUC: 0.8437
  MLP                  | F1: 0.6024 | Recall: 0.5314 | ROC-AUC: 0.8998
  MLP                  | F1: 0.6024 | Recall: 0.5314 | ROC-AUC: 0.8998
  XGBoost              | F1: 0.6535 | Recall: 0.6073 | ROC-AUC: 0.9302
  XGBoost              | F1: 0.6535 | Recall: 0.6073 | ROC-AUC: 0.93

## Paso 3.3: Comparación de Resultados

Identificaremos el mejor modelo para cada configuración (Baseline y cada nivel de SMOTE) y compararemos los ganadores.

### Resumen FASE 3: Balanceo de Clases y Selección de Modelo

En esta fase aplicamos SMOTE de forma incremental (5%, 10%, 15%) y evaluamos 8 modelos de clasificación. Los resultados muestran que:

**🎯 Hallazgos principales:**

1. **Efecto de SMOTE varía por modelo:**
   - Algunos modelos mejoran significativamente con SMOTE (ej: Random Forest, Decision Tree)
   - Otros funcionan mejor sin balanceo (ej: XGBoost, posiblemente por manejo interno de desbalanceo)
   - El nivel óptimo de SMOTE depende del algoritmo

2. **Trade-offs observados:**
   - SMOTE generalmente mejora **Recall** (detecta más compradores)
   - Puede reducir **Precision** (más falsos positivos)
   - El **F1-Score** captura el balance entre ambos

3. **Consideraciones prácticas:**
   - Tiempo de entrenamiento aumenta con SMOTE (más datos)
   - Modelos ensemble (RF, XGBoost) son más robustos
   - Redes neuronales (MLP) pueden requerir más datos o ajuste fino

**✅ Recomendación final:**
El modelo óptimo identificado será usado en la siguiente fase para optimización de hiperparámetros.

In [ ]:
# Identificar top 3 configuraciones globales
todas_configuraciones = []

# Baseline
for model_name in baseline_results.keys():
    todas_configuraciones.append({
        'Configuración': 'Baseline',
        'Modelo': model_name,
        'F1-Score': baseline_results[model_name]['F1-Score'],
        'Recall': baseline_results[model_name]['Recall'],
        'Precision': baseline_results[model_name]['Precision'],
        'ROC-AUC': baseline_results[model_name]['ROC-AUC'],
        'Accuracy': baseline_results[model_name]['Accuracy'],
        'Tiempo (s)': baseline_results[model_name]['Training Time (s)']
    })

# SMOTE
for smote_level in ['SMOTE 5%', 'SMOTE 10%', 'SMOTE 15%']:
    for model_name in smote_results[smote_level].keys():
        todas_configuraciones.append({
            'Configuración': smote_level,
            'Modelo': model_name,
            'F1-Score': smote_results[smote_level][model_name]['F1-Score'],
            'Recall': smote_results[smote_level][model_name]['Recall'],
            'Precision': smote_results[smote_level][model_name]['Precision'],
            'ROC-AUC': smote_results[smote_level][model_name]['ROC-AUC'],
            'Accuracy': smote_results[smote_level][model_name]['Accuracy'],
            'Tiempo (s)': smote_results[smote_level][model_name]['Training Time (s)']
        })

df_todas = pd.DataFrame(todas_configuraciones)
top_3 = df_todas.nlargest(3, 'F1-Score')

print("\n" + "🥇🥈🥉" * 30)
print("TOP 3 MEJORES CONFIGURACIONES (Modelo + SMOTE)")
print("🥇🥈🥉" * 30)
print(top_3.round(4).to_string(index=False))

# Análisis comparativo del top 3
print("\n" + "=" * 100)
print("ANÁLISIS COMPARATIVO DEL TOP 3")
print("=" * 100)

for idx, (i, row) in enumerate(top_3.iterrows(), 1):
    medalla = ["🥇", "🥈", "🥉"][idx-1]
    print(f"\n{medalla} PUESTO {idx}: {row['Modelo']} ({row['Configuración']})")
    print(f"  ├─ F1-Score:  {row['F1-Score']:.4f}")
    print(f"  ├─ Recall:    {row['Recall']:.4f}")
    print(f"  ├─ Precision: {row['Precision']:.4f}")
    print(f"  ├─ ROC-AUC:   {row['ROC-AUC']:.4f}")
    print(f"  ├─ Accuracy:  {row['Accuracy']:.4f}")
    print(f"  └─ Tiempo:    {row['Tiempo (s)']:.2f}s")

### 3.3.5: Top 3 Modelos - Comparación Detallada

In [ ]:
# Analizar cómo cada modelo se comporta con diferentes niveles de SMOTE
print("\n" + "=" * 120)
print("EVOLUCIÓN DE F1-SCORE POR MODELO SEGÚN NIVEL DE SMOTE")
print("=" * 120)

# Obtener lista de modelos
all_models = list(baseline_results.keys())

evolution_data = []

for model_name in all_models:
    row = {
        'Modelo': model_name,
        'Baseline': baseline_results[model_name]['F1-Score'],
        'SMOTE 5%': smote_results['SMOTE 5%'][model_name]['F1-Score'],
        'SMOTE 10%': smote_results['SMOTE 10%'][model_name]['F1-Score'],
        'SMOTE 15%': smote_results['SMOTE 15%'][model_name]['F1-Score']
    }
    
    # Calcular cambio respecto a baseline
    row['Δ Max (%)'] = ((max(row['SMOTE 5%'], row['SMOTE 10%'], row['SMOTE 15%']) - row['Baseline']) / row['Baseline'] * 100)
    
    evolution_data.append(row)

df_evolution = pd.DataFrame(evolution_data)
df_evolution = df_evolution.sort_values('Baseline', ascending=False)

print(df_evolution.round(4).to_string(index=False))

# Identificar modelos que mejoran con SMOTE
print("\n📊 ANÁLISIS:")
mejoran_con_smote = df_evolution[df_evolution['Δ Max (%)'] > 0].sort_values('Δ Max (%)', ascending=False)
empeoran_con_smote = df_evolution[df_evolution['Δ Max (%)'] <= 0].sort_values('Δ Max (%)')

print(f"\n✅ Modelos que MEJORAN con SMOTE ({len(mejoran_con_smote)}):")
for idx, row in mejoran_con_smote.iterrows():
    print(f"  • {row['Modelo']}: +{row['Δ Max (%)']:.2f}% mejora")

print(f"\n❌ Modelos que NO mejoran con SMOTE ({len(empeoran_con_smote)}):")
for idx, row in empeoran_con_smote.iterrows():
    print(f"  • {row['Modelo']}: {row['Δ Max (%)']:.2f}% cambio")

### 3.3.4: Análisis del Efecto de SMOTE por Modelo

In [ ]:
# Crear tabla comparativa de los ganadores
ganadores = []

# Agregar baseline
ganadores.append({
    'Configuración': 'Baseline (Sin SMOTE)',
    'Modelo': best_baseline,
    'F1-Score': baseline_df.loc[best_baseline, 'F1-Score'],
    'Recall': baseline_df.loc[best_baseline, 'Recall'],
    'Precision': baseline_df.loc[best_baseline, 'Precision'],
    'ROC-AUC': baseline_df.loc[best_baseline, 'ROC-AUC'],
    'Tiempo (s)': baseline_df.loc[best_baseline, 'Training Time (s)']
})

# Agregar cada nivel de SMOTE
for smote_level, data in best_by_smote.items():
    ganadores.append({
        'Configuración': smote_level,
        'Modelo': data['modelo'],
        'F1-Score': data['f1'],
        'Recall': data['recall'],
        'Precision': data['precision'],
        'ROC-AUC': data['roc_auc'],
        'Tiempo (s)': data['tiempo']
    })

df_ganadores = pd.DataFrame(ganadores)

print("\n" + "=" * 120)
print("COMPARACIÓN DE MODELOS GANADORES POR CONFIGURACIÓN")
print("=" * 120)
print(df_ganadores.round(4).to_string(index=False))

# Identificar el ganador absoluto
mejor_absoluto_idx = df_ganadores['F1-Score'].idxmax()
mejor_config = df_ganadores.loc[mejor_absoluto_idx, 'Configuración']
mejor_modelo = df_ganadores.loc[mejor_absoluto_idx, 'Modelo']

print("\n" + "🎯" * 60)
print("MODELO ÓPTIMO GENERAL")
print("🎯" * 60)
print(f"Configuración: {mejor_config}")
print(f"Modelo: {mejor_modelo}")
print(f"F1-Score: {df_ganadores.loc[mejor_absoluto_idx, 'F1-Score']:.4f}")
print(f"Recall: {df_ganadores.loc[mejor_absoluto_idx, 'Recall']:.4f}")
print(f"Precision: {df_ganadores.loc[mejor_absoluto_idx, 'Precision']:.4f}")
print(f"ROC-AUC: {df_ganadores.loc[mejor_absoluto_idx, 'ROC-AUC']:.4f}")
print(f"Tiempo de entrenamiento: {df_ganadores.loc[mejor_absoluto_idx, 'Tiempo (s)']:.2f}s")
print("🎯" * 60)

### 3.3.3: Comparación de los Ganadores

In [ ]:
# Analizar cada nivel de SMOTE
best_by_smote = {}

for smote_level in ['SMOTE 5%', 'SMOTE 10%', 'SMOTE 15%']:
    print("\n" + "=" * 100)
    print(f"RESULTADOS {smote_level} - Ordenados por F1-Score")
    print("=" * 100)
    
    # Crear DataFrame para este nivel
    smote_df = pd.DataFrame(smote_results[smote_level]).T
    smote_df = smote_df.sort_values('F1-Score', ascending=False)
    
    print(smote_df.round(4).to_string())
    
    # Identificar el mejor
    best_model = smote_df.index[0]
    best_by_smote[smote_level] = {
        'modelo': best_model,
        'f1': smote_df.loc[best_model, 'F1-Score'],
        'recall': smote_df.loc[best_model, 'Recall'],
        'precision': smote_df.loc[best_model, 'Precision'],
        'roc_auc': smote_df.loc[best_model, 'ROC-AUC'],
        'tiempo': smote_df.loc[best_model, 'Training Time (s)']
    }
    
    print(f"\n🏆 Mejor modelo {smote_level}: {best_model}")
    print(f"  • F1-Score: {best_by_smote[smote_level]['f1']:.4f}")
    print(f"  • Recall: {best_by_smote[smote_level]['recall']:.4f}")
    print(f"  • Precision: {best_by_smote[smote_level]['precision']:.4f}")
    print(f"  • ROC-AUC: {best_by_smote[smote_level]['roc_auc']:.4f}")

### 3.3.2: Mejor Modelo por Nivel de SMOTE

In [ ]:
# Crear DataFrame con resultados baseline
baseline_df = pd.DataFrame(baseline_results).T
baseline_df = baseline_df[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'Training Time (s)']]
baseline_df = baseline_df.sort_values('F1-Score', ascending=False)

print("=" * 100)
print("RESULTADOS BASELINE (SIN SMOTE) - Ordenados por F1-Score")
print("=" * 100)
print(baseline_df.round(4).to_string())

# Identificar el mejor
best_baseline = baseline_df.index[0]
print("\n" + "🏆" * 50)
print(f"MEJOR MODELO BASELINE: {best_baseline}")
print(f"  • F1-Score: {baseline_df.loc[best_baseline, 'F1-Score']:.4f}")
print(f"  • Recall: {baseline_df.loc[best_baseline, 'Recall']:.4f}")
print(f"  • Precision: {baseline_df.loc[best_baseline, 'Precision']:.4f}")
print(f"  • ROC-AUC: {baseline_df.loc[best_baseline, 'ROC-AUC']:.4f}")
print(f"  • Tiempo de entrenamiento: {baseline_df.loc[best_baseline, 'Training Time (s)']:.2f}s")
print("🏆" * 50)

### 3.3.1: Mejor Modelo Baseline (Sin SMOTE)